In [3]:
import os

# Add the directory containing the executable to the PATH
os.environ["PATH"] += os.pathsep + "/usr/local/cuda/bin"

# Check if the directory is added to the a
print(os.environ["PATH"])

/opt/tljh/user/bin:/bin:/usr/bin:/usr/local/cuda/bin


In [4]:
%%bash
nvcc --version
nvprof --version
nsys --version
ncu --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2025 NVIDIA Corporation
Built on Wed_Apr__9_19:24:57_PDT_2025
Cuda compilation tools, release 12.9, V12.9.41
Build cuda_12.9.r12.9/compiler.35813241_0
nvprof: NVIDIA (R) Cuda command line profiler
Copyright (c) 2012 - 2025 NVIDIA Corporation
Release version 12.9.19 (21)
NVIDIA Nsight Systems version 2025.1.3.140-251335620677v0
NVIDIA (R) Nsight Compute Command Line Profiler
Copyright (c) 2018-2025 NVIDIA Corporation
Version 2025.2.0.0 (build 35613519) (public-release)


In [5]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Nov 24 13:37:59 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 575.51.03              Driver Version: 575.51.03      CUDA Version: 12.9     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla V100-PCIE-32GB           Off |   00000000:00:10.0 Off |                    0 |
| N/A   27C    P0             22W /  250W |       0MiB /  32768MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [14]:
%%writefile kmean_gpu.cu

#include <cuda_runtime.h>
#include <cstdio>
#include <vector>
#include <random>
#include <algorithm>
#include <numeric>
#include <cmath>
#include <cassert>
#include <unordered_set>
#include <limits>
#include <cfloat>

// ---------------- Config ----------------
static const int DIM      = 64;
static const float ALPHA  = 0.7f;
static const int SEED     = 2025; 
static const int N        = 1 << 20; // 1 Million vectors
static const int K        = 2048;    // Clusters
static const int NPROBE   = 32;      // Search depth
static const int TOPK     = 5;
static const int KMEANS_ITERS = 15;  // K-Means iterations
static int BLOCK = 512;


using Vec = std::vector<float>;

// ---------------- Embedding Generator ----------------

static Vec numberBase[76]; // 1..75
static Vec posBase[25];    // 0..24

static void normInPlace(Vec &v) {
    double s = 0;
    for (float x : v) s += (double)x * x;
    float n = float(std::sqrt(s) + 1e-12);
    for (float &x : v) x /= n;
}

static Vec randUnit(std::mt19937 &rng) {
    std::uniform_real_distribution<float> U(-1.f, 1.f);
    Vec v(DIM);
    for (int i = 0; i < DIM; i++) v[i] = U(rng);
    normInPlace(v);
    return v;
}

static void initBases() {
    std::mt19937 rng(SEED);
    for (int n = 1; n <= 75; n++) numberBase[n] = randUnit(rng);
    for (int i = 0; i < 25; i++)  posBase[i]    = randUnit(rng);
}

static Vec cardToVec(const int card[25]) {
    Vec out(DIM, 0.f);
    for (int i = 0; i < 25; i++) {
        int n = card[i];
        const Vec &b = numberBase[n];
        const Vec &p = posBase[i];
        for (int j = 0; j < DIM; j++)
            out[j] += b[j] + ALPHA * p[j];
    }
    normInPlace(out);
    return out;
}

static void genCard(std::mt19937 &rng, int out[25]) {
    std::vector<int> p(75);
    std::iota(p.begin(), p.end(), 1);
    std::shuffle(p.begin(), p.end(), rng);
    for (int i = 0; i < 25; i++) out[i] = p[i];
}

static double dot_host(const float* a, const float* b) {
    double s = 0;
    for (int i = 0; i < DIM; i++) s += (double)a[i] * b[i];
    return s;
}

// ---------------- GPU K-Means Kernels ----------------

// E-step: Assign data to nearest centroid (E-Step: Data Assignment)
__global__ void assignAndAccumulateKernel(const float* data,      // [Input] Dataset (N * DIM)
                                          int N,                 // Total number of data points
                                          const float* centroids, // [Input] Centroids (K * DIM)
                                          int K,                 // Total number of clusters K
                                          int* assign,            // [Output] Assignment results (N elements, storing 0..K-1)
                                          double* sums,           // [Output] Accumulator: Centroid vector sums (K * DIM, using double precision)
                                          int* counts)           // [Output] Accumulator: Centroid counts (K elements)
{
    // Each thread processes one or more data points
    for (int i = blockIdx.x * blockDim.x + threadIdx.x;
         i < N;
         i += gridDim.x * blockDim.x) {

        // Get the starting pointer for the current data point xi
        const float* xi = data + (size_t)i * DIM;

        int bestC = 0;
        // Initialize the minimum distance with a sufficiently large number (e.g., FLT_MAX)
        float bestD = 1e30f; 

        // Iterate through all K centroids to find the closest one
        for (int c = 0; c < K; ++c) {
            // Get the starting pointer for the current centroid ctr
            const float* ctr = centroids + (size_t)c * DIM;
            
            // Use double precision for dot product accumulation to improve numerical stability
            double dot_double = 0.0; 
            for (int d = 0; d < DIM; ++d) {
                // Promote float values to double for multiplication and addition
                dot_double += (double)xi[d] * (double)ctr[d]; 
            }
            float dot = (float)dot_double; // Convert the final result back to float for distance comparison
            
            // Calculate cosine distance: 1.0 - similarity (similarity is the dot product)
            float dist = 1.f - dot;
            
            // Update the nearest centroid
            if (dist < bestD) {
                bestD = dist;
                bestC = c;
            }
        }

        // Record the assignment result for data point i
        assign[i] = bestC;

        // Atomic add to accumulators (Atomic operations resolve concurrent write conflicts)
        // Accumulate count
        atomicAdd(&counts[bestC], 1);
        
        // Accumulate vector sum
        size_t base = (size_t)bestC * DIM;
        for (int d = 0; d < DIM; ++d) {
            // Convert the float xi[d] to double before atomically adding to the double sums array
            atomicAdd(&sums[base + d], (double)xi[d]); 
        }
    }
}

// -----------------------------------------------------------------------------

// M-step: Update centroids (M-Step: Centroid Update)
__global__ void updateCentroidsKernel(float* centroids,      // [Output] Centroids (updated values)
                                      const double* sums,    // [Input] Accumulated vector sums (double precision)
                                      const int* counts,     // [Input] Accumulated counts
                                      int K) {
    // Each thread processes one centroid c
    int c = blockIdx.x * blockDim.x + threadIdx.x;
    if (c >= K) return;

    int cnt = counts[c];
    float* ctr = centroids + (size_t)c * DIM;      // Pointer to the centroid to be updated
    const double* sumc = sums + (size_t)c * DIM;   // Pointer to the vector sum for centroid c

    if (cnt > 0) { // Only update centroids that were assigned data points
        double norm2 = 0.0;
        for (int d = 0; d < DIM; ++d) {
            // FIX: Use double for precise average calculation (sum / count)
            double v_double = sumc[d] / (double)cnt;
            float v = (float)v_double; // Convert result back to float for storage
            
            ctr[d] = v;
            
            // Use double precision to accumulate the squared norm, preparing for unit normalization
            norm2 += v_double * v_double; 
        }
        
        // Centroid Normalization (Unit Length)
        // Calculate the L2 norm (magnitude)
        float n = float(std::sqrt(norm2) + 1e-12); 
        
        // Normalize the centroid vector onto the unit sphere
        for (int d = 0; d < DIM; ++d) {
            ctr[d] /= n;
        }
    }
}


// ---------------- Host: Inverted Lists ----------------

static void buildInvertedLists(
    const std::vector<int>& assign,
    int N, int K,
    std::vector<std::vector<int>>& lists
) {
    lists.assign(K, {});
    for (int i = 0; i < N; ++i) {
        int c = assign[i];
        if (c >= 0 && c < K) {
            lists[c].push_back(i);
        }
    }
}



// ---------------- Main ----------------
int main() {

    cudaDeviceProp prop;
    cudaGetDeviceProperties(&prop, 0);

    int smCount = prop.multiProcessorCount;
    int gridSizeN = smCount * 4;

    printf("GPU: %s\n", prop.name);
    printf("SM count: %d\n", prop.multiProcessorCount);
    printf("Params: N=%d  K=%d  nprobe=%d  TOPK=%d  DIM=%d  KMEANS_ITERS=%d\n",
           N, K, NPROBE, TOPK, DIM, KMEANS_ITERS);
    printf("Mode: GPU Training -> CPU Search\n");

    initBases();
    std::mt19937 rng(SEED + 7);

    // 1) Data Generation
    std::vector<int>  h_cards((size_t)N * 25);
    std::vector<float> h_data((size_t)N * DIM);

    printf("[INIT] Generating %d vectors...\n", N);
    for (int i = 0; i < N; i++) {
        int c[25];
        genCard(rng, c);
        for (int t = 0; t < 25; ++t) h_cards[(size_t)i * 25 + t] = c[t];
        Vec v = cardToVec(c);
        for (int d = 0; d < DIM; ++d) h_data[(size_t)i * DIM + d] = v[d];
    }

    // 2) Build Query
    int qc[25];
    for (int t = 0; t < 25; ++t) qc[t] = h_cards[t]; // Copy 0-th card
    qc[3] = 75; qc[17] = 1; std::swap(qc[5], qc[19]); // Modify it
    Vec qvec = cardToVec(qc);
    std::vector<float> q_host(qvec.begin(), qvec.end());

    {
        double dot0 = dot_host(q_host.data(), &h_data[0]);
        printf("[DEBUG] Query vs Data[0]: dot=%.9f dist=%.9f\n", dot0, 1.0 - dot0);
    }

    // 3) GPU Allocations for Training
    float *d_data = 0, *d_centroids = 0;
    int   *d_assign = 0, *d_counts = 0;
    double *d_sums_double = 0;

    cudaMalloc(&d_data,      (size_t)N * DIM * sizeof(float));
    cudaMalloc(&d_centroids, (size_t)K * DIM * sizeof(float));
    cudaMalloc(&d_assign,    N * sizeof(int));
    cudaMalloc(&d_sums_double, (size_t)K * DIM * sizeof(double));
    cudaMalloc(&d_counts,    K * sizeof(int));

    cudaMemcpy(d_data, h_data.data(), (size_t)N * DIM * sizeof(float), cudaMemcpyHostToDevice);

    // 4) Initialize Centroids (Random select from data)
    {
        std::vector<int> idx(N);
        std::iota(idx.begin(), idx.end(), 0);
        std::shuffle(idx.begin(), idx.end(), rng);
        std::vector<float> h_initC((size_t)K * DIM);
        for (int c = 0; c < K; ++c) {
            int i = idx[c];
            std::copy_n(&h_data[(size_t)i * DIM], DIM, &h_initC[(size_t)c * DIM]);
        }
        cudaMemcpy(d_centroids, h_initC.data(), (size_t)K * DIM * sizeof(float), cudaMemcpyHostToDevice);
    }

    // 5) GPU K-Means Training
    {
        dim3 block(BLOCK);
        dim3 gridN(gridSizeN);

        printf("[TRAIN] Running K-Means on GPU (%d iters)...\n", KMEANS_ITERS);

        for (int it = 0; it < KMEANS_ITERS; ++it) {
            cudaMemset(d_sums_double,  0, (size_t)K * DIM * sizeof(double)); 
            cudaMemset(d_counts, 0, K * sizeof(int));

            assignAndAccumulateKernel<<<gridN,block>>>(
                d_data, N, d_centroids, K, d_assign, d_sums_double, d_counts
            );
            cudaDeviceSynchronize();

            updateCentroidsKernel<<<gridN,block>>>(
                d_centroids, d_sums_double, d_counts, K
            );
            cudaDeviceSynchronize();
        }
    }

    // 6) Retrieve Training Results
    std::vector<int> h_assign(N);
    cudaMemcpy(h_assign.data(), d_assign, N * sizeof(int), cudaMemcpyDeviceToHost);

    // Retrieve final centroids for CPU search
    std::vector<float> h_finalCentroids(K * DIM);
    cudaMemcpy(h_finalCentroids.data(), d_centroids, (size_t)K * DIM * sizeof(float), cudaMemcpyDeviceToHost);

    // We are done with GPU memory
    cudaFree(d_data);
    cudaFree(d_centroids);
    cudaFree(d_assign);
    cudaFree(d_sums_double);
    cudaFree(d_counts);

    // 7) Build IVF Index (Host)
    printf("[INDEX] Building Inverted Lists on Host...\n");
    std::vector<std::vector<int>> lists;
    buildInvertedLists(h_assign, N, K, lists);

    int nonEmpty = 0;
    for(const auto& list : lists) if(!list.empty()) nonEmpty++;
    printf("[INDEX] Non-empty clusters: %d / %d\n", nonEmpty, K);

    // ---------------------------------------------------------
    //  SEARCH PHASE (CPU)
    // ---------------------------------------------------------
    printf("\n[SEARCH] CPU Search (nprobe=%d)...\n", NPROBE);

    // Step A: Coarse Search (Find nearest NPROBE clusters)
    // Format: {distance, cluster_id}
    std::vector<std::pair<float, int>> centerDists;
    centerDists.reserve(K);

    for (int c = 0; c < K; ++c) {
        const float* ctr = &h_finalCentroids[(size_t)c * DIM];
        double dotVal = dot_host(q_host.data(), ctr);
        float dist = 1.0f - (float)dotVal;
        centerDists.push_back({dist, c});
    }

    // Sort centroids by distance ASC
    std::sort(centerDists.begin(), centerDists.end(), 
              [](const std::pair<float, int>& a, const std::pair<float, int>& b){
                  return a.first < b.first;
              });

    // Step B: Gather Candidates & Exact Search
    // Format: {distance, vector_id}
    std::vector<std::pair<float, int>> candidates;
    // Reserve some memory to avoid reallocations (heuristic)
    candidates.reserve((N / K) * NPROBE * 2);

    int visitedVecs = 0;
    for (int i = 0; i < NPROBE && i < K; ++i) {
        int c_id = centerDists[i].second;
        const auto& bucket = lists[c_id];
        visitedVecs += bucket.size();

        for (int vecIdx : bucket) {
            const float* vec = &h_data[(size_t)vecIdx * DIM];
            double dotVal = dot_host(q_host.data(), vec);
            float dist = 1.0f - (float)dotVal;
            candidates.push_back({dist, vecIdx});
        }
    }

    printf("[SEARCH] Scanned %d vectors from top %d clusters.\n", visitedVecs, NPROBE);

    // Step C: Ranking (Top-K)
    if (candidates.empty()) {
        printf("No candidates found.\n");
    } else {
        int finalK = std::min((int)candidates.size(), TOPK);
        
        // Partial sort gives us the smallest K elements at the beginning
        std::partial_sort(candidates.begin(), 
                          candidates.begin() + finalK, 
                          candidates.end(),
                          [](const std::pair<float, int>& a, const std::pair<float, int>& b){
                              return a.first < b.first;
                          });

        printf("\nTop-%d Results:\n", finalK);
        for (int i = 0; i < finalK; ++i) {
            printf("%2d) id=%d  dist=%.6f  sim=%.6f\n",
                   i+1, candidates[i].second, candidates[i].first, 1.f - candidates[i].first);
        }
    }

    return 0;
}

Overwriting kmean_gpu.cu


In [23]:
%%bash
nvcc -O3 -std=c++17 -arch=sm_70 kmean_gpu.cu -o kmean_gpu -Wno-deprecated-gpu-targets

In [24]:
%%bash
nvprof ./kmean_gpu

==386485== NVPROF is profiling process 386485, command: ./kmean_gpu


GPU: Tesla V100-PCIE-32GB
SM count: 80
Params: N=1048576  K=2048  nprobe=32  TOPK=5  DIM=64  KMEANS_ITERS=15
Mode: GPU Training -> CPU Search
[INIT] Generating 1048576 vectors...
[DEBUG] Query vs Data[0]: dot=0.927471359 dist=0.072528641
[TRAIN] Running K-Means on GPU (15 iters)...
[INDEX] Building Inverted Lists on Host...
[INDEX] Non-empty clusters: 2048 / 2048

[SEARCH] CPU Search (nprobe=32)...
[SEARCH] Scanned 18343 vectors from top 32 clusters.

Top-5 Results:
 1) id=0  dist=0.072529  sim=0.927471
 2) id=89698  dist=0.157913  sim=0.842087
 3) id=913782  dist=0.158968  sim=0.841032
 4) id=350035  dist=0.164414  sim=0.835586
 5) id=56801  dist=0.177664  sim=0.822336


==386485== Profiling application: ./kmean_gpu
==386485== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   99.57%  55.2780s        15  3.68520s  3.67646s  3.69604s  assignAndAccumulateKernel(float const *, int, float const *, int, int*, double*, int*)
                    0.42%  233.02ms         2  116.51ms  96.160us  232.92ms  [CUDA memcpy HtoD]
                    0.00%  2.0804ms         2  1.0402ms  113.41us  1.9669ms  [CUDA memcpy DtoH]
                    0.00%  2.0160ms        15  134.40us  133.25us  135.10us  updateCentroidsKernel(float*, double const *, int const *, int)
                    0.00%  60.928us        30  2.0300us  1.3440us  4.4800us  [CUDA memset]
      API calls:   98.51%  55.2807s        30  1.84269s  94.913us  3.69610s  cudaDeviceSynchronize
                    1.05%  591.17ms         5  118.23ms  5.4120us  590.38ms  cudaMalloc
                    0.42%  238.21ms         4  59.551ms  486.11us  2

In [20]:
%%bash
nsys profile -o kmean_gpu ./kmean_gpu

         This may increase runtime overhead and the likelihood of false
         dependencies across CUDA Streams. If you wish to avoid this, please
         disable the feature with --cuda-event-trace=false.
Try the 'nsys status --environment' command to learn more.

Try the 'nsys status --environment' command to learn more.



Params: N=16777216  K=1024  nprobe=32  TOPK=5  DIM=64  KMEANS_ITERS=15
Mode: GPU Training -> CPU Search
[INIT] Generating 16777216 vectors...
[DEBUG] Query vs Data[0]: dot=0.927471359 dist=0.072528641
[TRAIN] Running K-Means on GPU (15 iters)...
[INDEX] Building Inverted Lists on Host...
[INDEX] Non-empty clusters: 1024 / 1024

[SEARCH] CPU Search (nprobe=32)...
[SEARCH] Scanned 557455 vectors from top 32 clusters.

Top-5 Results:
 1) id=0  dist=0.072529  sim=0.927471
 2) id=15528589  dist=0.136097  sim=0.863903
 3) id=13929498  dist=0.139304  sim=0.860696
 4) id=8185180  dist=0.146311  sim=0.853689
 5) id=16494452  dist=0.150325  sim=0.849675
Generating '/tmp/nsys-report-bf67.qdstrm'
[1/1] [========================100%] kmean_gpu.nsys-rep
Generated:
	/home/jupyter-feifan_chen@dlsu.e-15ebb/kmean_gpu.nsys-rep


In [19]:
%%writefile kmean_cpu.cpp

#include <cstdio>
#include <vector>
#include <random>
#include <algorithm>
#include <numeric>
#include <cmath>
#include <cassert>
#include <unordered_set>
#include <limits>
#include <cfloat>
#include <cstring>

// ---------------- Config ----------------
static const int DIM          = 64;
static const float ALPHA      = 0.7f;
static const int SEED         = 2025; 
static const int N            = 1 << 14; // demo: 16384
static const int K            = 1024;
static const int NPROBE       = 4;       // cluster depth during search
static const int TOPK         = 5;

static const int KMEANS_ITERS = 15;      // K-Means iters

using Vec = std::vector<float>;

// ---------------- Embedding basic ----------------

static Vec numberBase[76]; // 1..75
static Vec posBase[25];    // 0..24

static void normInPlace(Vec &v) {
    double s = 0;
    for (float x : v) s += (double)x * x;
    float n = float(std::sqrt(s) + 1e-12);
    for (float &x : v) x /= n;
}

static Vec randUnit(std::mt19937 &rng) {
    std::uniform_real_distribution<float> U(-1.f, 1.f);
    Vec v(DIM);
    for (int i = 0; i < DIM; i++) v[i] = U(rng);
    normInPlace(v);
    return v;
}

static void initBases() {
    std::mt19937 rng(SEED);
    for (int n = 1; n <= 75; n++) numberBase[n] = randUnit(rng);
    for (int i = 0; i < 25; i++)  posBase[i]    = randUnit(rng);
}

static Vec cardToVec(const int card[25]) {
    Vec out(DIM, 0.f);
    for (int i = 0; i < 25; i++) {
        int n = card[i];
        const Vec &b = numberBase[n];
        const Vec &p = posBase[i];
        for (int j = 0; j < DIM; j++)
            out[j] += b[j] + ALPHA * p[j];
    }
    normInPlace(out);
    return out;
}

static void genCard(std::mt19937 &rng, int out[25]) {
    std::vector<int> p(75);
    std::iota(p.begin(), p.end(), 1);
    std::shuffle(p.begin(), p.end(), rng);
    for (int i = 0; i < 25; i++) out[i] = p[i];
}

static double dot_host(const float* a, const float* b) {
    double s = 0;
    for (int i = 0; i < DIM; i++) s += (double)a[i] * b[i];
    return s;
}

// ---------------- Device: Distance Kernels (Simulated for Training) ----------------

/**
 * E-Step: Assign data to nearest centroid & Accumulate stats
 * * This function simulates the "Expectation" step of K-Means.
 * It iterates through every data point, finds the closest centroid (using Cosine Distance),
 * assigns the point to that cluster, and accumulates the vector sums and counts
 * needed for the next update step.
 */
void assignAndAccumulateKernel(const float* data,
                                  int N_points,
                                  const float* centroids,
                                  int K_clusters,
                                  int* assign,
                                  float* sums,
                                  int* counts) {
    
    // Loop 1: Iterate through every single data point (0 to N-1)
    for (int i = 0; i < N_points; i++) {
        
        // Get pointer to the current data vector (Dimension = DIM)
        const float* xi = data + (size_t)i * DIM;

        int bestC = 0;          // Store the index of the nearest cluster
        float bestD = 1e30f;    // Initialize minimum distance to a large value

        // Loop 2: Compare current point 'xi' against all K centroids
        for (int c = 0; c < K_clusters; ++c) {
            const float* ctr = centroids + (size_t)c * DIM;
            
            // Calculate Dot Product (Inner Product)
            float dot = 0.f;
            for (int d = 0; d < DIM; ++d) dot += xi[d] * ctr[d];

            // Convert Cosine Similarity to Cosine Distance
            // Distance = 1.0 - Similarity
            float dist = 1.f - dot;

            // Keep track of the nearest centroid found so far
            if (dist < bestD) {
                bestD = dist;
                bestC = c;
            }
        }

        // 1. Record the assignment: Point 'i' belongs to Cluster 'bestC'
        assign[i] = bestC;

        // 2. Accumulate count: Increment the member count for this cluster
        counts[bestC]++;

        // 3. Accumulate sums: Add this vector's coordinates to the cluster's total
        // This prepares for the average calculation in the next step.
        size_t base = (size_t)bestC * DIM;
        for (int d = 0; d < DIM; ++d) {
            sums[base + d] += xi[d];
        }
    }
}

/**
 * M-Step: Update Centroids
 * * This function simulates the "Maximization" step of K-Means.
 * It calculates the new position of each centroid by averaging the vectors 
 * assigned to it, and then normalizes the result to ensure it stays on the unit hypersphere.
 */
void updateCentroidsKernel(float* centroids,
                                  const float* sums,
                                  const int* counts,
                                  int K_clusters) {
    
    // Loop 1: Iterate through every cluster (0 to K-1)
    for (int c = 0; c < K_clusters; c++) {
        
        int cnt = counts[c];    // Number of points in this cluster
        
        // Pointers to the current centroid and its accumulated sum
        float* ctr = centroids + (size_t)c * DIM;
        const float* sumc = sums + (size_t)c * DIM;
    
        // Only update if the cluster is not empty
        if (cnt > 0) {
            double norm2 = 0.0;

            // Loop 2: Calculate the Mean (Average) Vector
            for (int d = 0; d < DIM; ++d) {
                // New Coordinate = Total Sum / Count
                float v = sumc[d] / (float)cnt;
                ctr[d] = v;

                // Accumulate squared magnitude for normalization later
                norm2 += (double)v * (double)v;
            }

            // Calculate the L2 Norm (Euclidean length)
            // Added 1e-12 to prevent division by zero
            float n = float(std::sqrt(norm2) + 1e-12);

            // Loop 3: Normalization
            // Since we use Cosine Distance, centroids must be normalized 
            // (length = 1.0) to lie on the unit sphere.
            for (int d = 0; d < DIM; ++d) {
                ctr[d] /= n;
            }
        }
    }
}

// ---------------- Host: Inverted Lists ----------------

static void buildInvertedLists(
    const std::vector<int>& assign,
    int N_points, int K_clusters,
    std::vector<std::vector<int>>& lists
) {
    lists.assign(K_clusters, {});
    for (int i = 0; i < N_points; ++i) {
        int c = assign[i];
        if (c >= 0 && c < K_clusters) {
            lists[c].push_back(i);
        }
    }
}

// ---------------- Main ----------------
int main() {
    printf("Params: N=%d  K=%d  nprobe=%d  TOPK=%d  DIM=%d  KMEANS_ITERS=%d\n",
           N, K, NPROBE, TOPK, DIM, KMEANS_ITERS);

    initBases();
    std::mt19937 rng(SEED + 7);

    clock_t start, end;
    double elapse = 0.0f;

    // 1) Building dataset
    std::vector<int>   h_cards((size_t)N * 25);
    std::vector<float> h_data((size_t)N * DIM);

    for (int i = 0; i < N; i++) {
        int c[25];
        genCard(rng, c);
        for (int t = 0; t < 25; ++t) h_cards[(size_t)i * 25 + t] = c[t];
        Vec v = cardToVec(c);
        for (int d = 0; d < DIM; ++d) h_data[(size_t)i * DIM + d] = v[d];
    }

    // 2) Build query
    int qc[25];
    for (int t = 0; t < 25; ++t) qc[t] = h_cards[t];
    qc[3] = 75; qc[17] = 1; std::swap(qc[5], qc[19]);
    Vec qvec = cardToVec(qc);
    std::vector<float> qhost(qvec.begin(), qvec.end());

    {
        std::vector<float> d0(DIM);
        for (int i = 0; i < DIM; i++) d0[i] = h_data[i];
        double dot0 = dot_host(qhost.data(), d0.data());
        printf("[DEBUG #1] host dot(q, data[0])=%.9f  dist=%.9f\n", dot0, 1.0 - dot0);
    }

    // 3) Pseudo-GPU Buffers (Training)
    float *d_data = 0, *d_centroids = 0;
    d_data = (float*)malloc((size_t)N * DIM * sizeof(float));
    d_centroids = (float*)malloc((size_t)K * DIM * sizeof(float));

    memcpy(d_data, h_data.data(), (size_t)N * DIM * sizeof(float));

    // 4) Initial centroids
    {
        std::vector<int> idx(N);
        std::iota(idx.begin(), idx.end(), 0);
        std::shuffle(idx.begin(), idx.end(), rng);
        std::vector<float> h_initC((size_t)K * DIM);
        for (int c = 0; c < K; ++c) {
            int i = idx[c];
            std::copy_n(&h_data[(size_t)i * DIM], DIM, &h_initC[(size_t)c * DIM]);
        }
        memcpy(d_centroids, h_initC.data(), (size_t)K * DIM * sizeof(float));
    }

    int   *d_assign = (int*)malloc(N * sizeof(int));
    float *d_sums   = (float*)malloc((size_t)K * DIM * sizeof(float));
    int   *d_counts = (int*)malloc(K * sizeof(int));

    // 5) K-Means Training (Simulating GPU Kernel Loop)
    {
        start = clock();
        printf("[BUILD] K-Means: iters=%d\n", KMEANS_ITERS);

        for (int it = 0; it < KMEANS_ITERS; ++it) {
            memset(d_sums,   0, (size_t)K * DIM * sizeof(float));
            memset(d_counts, 0, K * sizeof(int));

            assignAndAccumulateKernel(d_data, N, d_centroids, K, d_assign, d_sums, d_counts);
            updateCentroidsKernel(d_centroids, d_sums, d_counts, K);
        }

        end = clock();
        double time_taken = ((double)(end-start))*1E3/CLOCKS_PER_SEC;
        elapse += time_taken;
        printf("K-means (C++ impl) time: %f ms\n", time_taken);
    }

    // 6) Build Inverted Index on Host
    //    Retrieve assignment from "device"
    std::vector<int> h_assign(N);
    memcpy(h_assign.data(), d_assign, N * sizeof(int));

    std::vector<std::vector<int>> lists;
    buildInvertedLists(h_assign, N, K, lists);

    int nonEmpty = 0;
    for (int c = 0; c < K; ++c) if (!lists[c].empty()) nonEmpty++;
    printf("[BUILD] Non-empty clusters: %d / %d\n", nonEmpty, K);

    // =================================================================================
    // CPU Search
    // =================================================================================

    start = clock();
    printf("\n[SEARCH] CPU Search logic (matching origin.cu, nprobe=%d)...\n", NPROBE);

    // Step A: Coarse Search (Find nearest NPROBE clusters)
    // Format: {distance, cluster_id}
    std::vector<std::pair<float, int>> centerDists;
    centerDists.reserve(K);

    for (int c = 0; c < K; ++c) {
        // d_centroids is a flat array, similar to how we used it in GPU code
        const float* ctr = d_centroids + (size_t)c * DIM; 
        double dotVal = dot_host(qhost.data(), ctr);
        float dist = 1.0f - (float)dotVal;
        centerDists.push_back({dist, c});
    }

    // Sort centroids by distance ASC
    std::sort(centerDists.begin(), centerDists.end());

    // Step B: Gather Candidates & Exact Search
    // Format: {distance, vector_id}
    std::vector<std::pair<float, int>> candidates;
    // Heuristic reserve
    candidates.reserve((N / K) * NPROBE * 2);

    int visitedVecs = 0;
    for (int i = 0; i < NPROBE && i < K; ++i) {
        int c_id = centerDists[i].second;
        const auto& bucket = lists[c_id];
        visitedVecs += (int)bucket.size();

        for (int vecIdx : bucket) {
            // Access raw data directly (Host DRAM)
            const float* vec = &h_data[(size_t)vecIdx * DIM];
            double dotVal = dot_host(qhost.data(), vec);
            float dist = 1.0f - (float)dotVal;
            candidates.push_back({dist, vecIdx});
        }
    }

    printf("[SEARCH] Scanned %d vectors from top %d clusters.\n", visitedVecs, NPROBE);

    // Step C: Ranking (Top-K)
    if (candidates.empty()) {
        printf("No candidates found.\n");
    } else {
        int finalK = std::min((int)candidates.size(), TOPK);
        
        // Use partial_sort like origin.cu
        std::partial_sort(candidates.begin(), 
                          candidates.begin() + finalK, 
                          candidates.end());

        printf("\nTop-%d Results:\n", finalK);
        for (int i = 0; i < finalK; ++i) {
            printf("%2d) id=%d  dist=%.6f  sim=%.6f\n",
                   i+1, candidates[i].second, candidates[i].first, 1.f - candidates[i].first);
        }

        // Verify with best (optional check)
        if (finalK > 0) {
            int bestId = candidates[0].second;
            const float* vec = &h_data[(size_t)bestId * DIM];
            double dotChk = dot_host(qhost.data(), vec);
            printf("[DEBUG] host check best id=%d  dot=%.9f  dist=%.9f\n",
                   bestId, dotChk, 1.0 - dotChk);
        }
    }

    end = clock();
    double search_time = ((double)(end-start))*1E3/CLOCKS_PER_SEC;
    printf("Function (Search Phase) time: %f ms\n", search_time);

    // Cleanup
    free(d_data);
    free(d_centroids);
    free(d_assign);
    free(d_sums);
    free(d_counts);

    return 0;
}

Overwriting kmean_cpu.cpp


In [2]:
%%bash
g++ -std=c++11 kmean_cpu.cpp -o kmean_cpu -lm

In [4]:
%%bash
./kmean_cpu

Params: N=16384  K=1024  nprobe=4  TOPK=5  DIM=64  KMEANS_ITERS=15
[DEBUG #1] host dot(q, data[0])=0.927471359  dist=0.072528641
[BUILD] K-Means: iters=15
K-means (C++ impl) time: 70167.580000 ms
[BUILD] Non-empty clusters: 1024 / 1024

[SEARCH] CPU Search logic (matching origin.cu, nprobe=4)...
[SEARCH] Scanned 105 vectors from top 4 clusters.

Top-5 Results:
 1) id=0  dist=0.072529  sim=0.927471
 2) id=15967  dist=0.177058  sim=0.822942
 3) id=1427  dist=0.216366  sim=0.783634
 4) id=3831  dist=0.226154  sim=0.773846
 5) id=10211  dist=0.246999  sim=0.753001
[DEBUG] host check best id=0  dot=0.927471359  dist=0.072528641
Function (Search Phase) time: 1.645000 ms


In [20]:
%%writefile kmean_gpu_share_mem.cu

#include <cuda_runtime.h>
#include <cstdio>
#include <vector>
#include <random>
#include <algorithm>
#include <numeric>
#include <cmath>
#include <cassert>
#include <cfloat>

// ---------------- Config ----------------
static const int DIM          = 64;
static const float ALPHA      = 0.7f;
static const int SEED         = 2025; 
static const int N            = 1 << 20; // 1 Million vectors
static const int K            = 2048;    // Clusters
static const int NPROBE       = 32;      // Search depth
static const int TOPK         = 5;
static const int KMEANS_ITERS = 15;      // K-Means iterations
static int BLOCK              = 512;     // Thread block size

// Tiling config for Shared Memory
// We load TILE_K centroids at a time into shared memory.
// 128 centroids * 64 floats * 4 bytes = 32 KB (Safe for essentially all GPUs)
#define TILE_K 128 

using Vec = std::vector<float>;

// ---------------- Helper for AtomicAdd Double ----------------
// Compute Capability < 6.0 needs a CAS loop for double atomicAdd.
#if !defined(__CUDA_ARCH__) || __CUDA_ARCH__ >= 600
#else
__device__ double atomicAdd(double* address, double val) {
    unsigned long long int* address_as_ull = (unsigned long long int*)address;
    unsigned long long int old = *address_as_ull, assumed;
    do {
        assumed = old;
        old = atomicCAS(address_as_ull, assumed,
                        __double_as_longlong(val + __longlong_as_double(assumed)));
    } while (assumed != old);
    return __longlong_as_double(old);
}
#endif

// ---------------- Embedding Generator ----------------
static Vec numberBase[76]; 
static Vec posBase[25];    

static void normInPlace(Vec &v) {
    double s = 0;
    for (float x : v) s += (double)x * x;
    float n = float(std::sqrt(s) + 1e-12);
    for (float &x : v) x /= n;
}

static Vec randUnit(std::mt19937 &rng) {
    std::uniform_real_distribution<float> U(-1.f, 1.f);
    Vec v(DIM);
    for (int i = 0; i < DIM; i++) v[i] = U(rng);
    normInPlace(v);
    return v;
}

static void initBases() {
    std::mt19937 rng(SEED);
    for (int n = 1; n <= 75; n++) numberBase[n] = randUnit(rng);
    for (int i = 0; i < 25; i++)  posBase[i]    = randUnit(rng);
}

static Vec cardToVec(const int card[25]) {
    Vec out(DIM, 0.f);
    for (int i = 0; i < 25; i++) {
        int n = card[i];
        const Vec &b = numberBase[n];
        const Vec &p = posBase[i];
        for (int j = 0; j < DIM; j++)
            out[j] += b[j] + ALPHA * p[j];
    }
    normInPlace(out);
    return out;
}

static void genCard(std::mt19937 &rng, int out[25]) {
    std::vector<int> p(75);
    std::iota(p.begin(), p.end(), 1);
    std::shuffle(p.begin(), p.end(), rng);
    for (int i = 0; i < 25; i++) out[i] = p[i];
}

static double dot_host(const float* a, const float* b) {
    double s = 0;
    for (int i = 0; i < DIM; i++) s += (double)a[i] * b[i];
    return s;
}

// ---------------- GPU K-Means Kernels (Optimized) ----------------

// E-Step:
// 1. Tiled Shared Memory loading for Centroids.
// 2. Per-Block Accumulation.
__global__ void assignAndAccumulatePerBlockKernel_V3(
                                         const float* data, int N,
                                         const float* centroids, int K,
                                         int* assign,
                                         double* block_sums,   // Output: (GridSize * K * DIM)
                                         int* block_counts) {  // Output: (GridSize * K)
    
    // Shared memory buffer to cache a tile of centroids
    __shared__ float sh_centroids[TILE_K * DIM];
    
    // Offsets for this specific block's accumulation buffer in global memory
    size_t block_sum_base = (size_t)blockIdx.x * (size_t)K * DIM;
    size_t block_count_base = (size_t)blockIdx.x * (size_t)K;

    // Grid-Stride Loop
    for (int i = blockIdx.x * blockDim.x + threadIdx.x;
         i < N;
         i += gridDim.x * blockDim.x) {

        const float* xi = data + (size_t)i * DIM;

        int bestC = 0;
        float bestD = 1e30f;
        
        // Loop over Centroids in chunks (Tiles)
        for (int k_tile = 0; k_tile < K; k_tile += TILE_K) {
            
            // --- Phase 1: Load Tile into Shared Memory ---
            int k_start = k_tile;
            
            // Cooperative loading: threads load distinct floats
            for (int t = threadIdx.x; t < TILE_K * DIM; t += blockDim.x) {
                int local_c = t / DIM;
                int local_d = t % DIM;
                int global_c = k_start + local_c;
                
                if (global_c < K) {
                    sh_centroids[t] = centroids[(size_t)global_c * DIM + local_d];
                } else {
                    sh_centroids[t] = 0.0f; // Padding
                }
            }
            __syncthreads(); // Wait for tile load to complete
            
            // --- Phase 2: Compute Distances against Tile ---
            int k_limit = (k_tile + TILE_K > K) ? (K - k_tile) : TILE_K;

            for (int c_local = 0; c_local < k_limit; ++c_local) {
                int c_global = k_start + c_local;
                
                float dot = 0.f;
                
                for (int d = 0; d < DIM; ++d) {
                    dot += xi[d] * sh_centroids[c_local * DIM + d];
                }
                
                float dist = 1.f - dot;
                if (dist < bestD) {
                    bestD = dist;
                    bestC = c_global;
                }
            }
            __syncthreads(); // Sync before loading next tile
        }

        assign[i] = bestC;

        // --- Accumulation ---
        // We atomicAdd to THIS block's specific buffer.
        atomicAdd(&block_counts[block_count_base + bestC], 1);

        size_t cluster_base = block_sum_base + (size_t)bestC * DIM;
        for (int d = 0; d < DIM; ++d) {
            atomicAdd(&block_sums[cluster_base + d], (double)xi[d]);
        }
    }
}

__global__ void reduceSumsKernel(double* final_sums, 
                                 int* final_counts, 
                                 const double* block_sums, 
                                 const int* block_counts, 
                                 int K, int GridSize) {
    
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    
    // Reduce Counts: 1 thread per cluster
    if (idx < K) {
        int total_cnt = 0;
        for (int b = 0; b < GridSize; ++b) {
            total_cnt += block_counts[(size_t)b * K + idx];
        }
        final_counts[idx] = total_cnt;
    }
    
    // Reduce Sums
    size_t total_elements = (size_t)K * DIM;
    for (size_t i = idx; i < total_elements; i += gridDim.x * blockDim.x) {
        double total_sum = 0.0;
        for (int b = 0; b < GridSize; ++b) {
            total_sum += block_sums[(size_t)b * total_elements + i];
        }
        final_sums[i] = total_sum;
    }
}

// M-step: Update centroids (Standard)
__global__ void updateCentroidsKernel(float* centroids,
                                      const double* sums, 
                                      const int* counts,
                                      int K) {
    int c = blockIdx.x * blockDim.x + threadIdx.x;
    if (c >= K) return;

    int cnt = counts[c];
    float* ctr = centroids + (size_t)c * DIM;
    const double* sumc = sums + (size_t)c * DIM; 

    if (cnt > 0) {
        double norm2 = 0.0;
        for (int d = 0; d < DIM; ++d) {
            double v_double = sumc[d] / (double)cnt; // Mean
            float v = (float)v_double;
            ctr[d] = v;
            norm2 += v_double * v_double;
        }
        // Normalize
        float n = float(std::sqrt(norm2) + 1e-12);
        for (int d = 0; d < DIM; ++d) {
            ctr[d] /= n;
        }
    }
}

// ---------------- Host Helpers ----------------

static void buildInvertedLists(
    const std::vector<int>& assign,
    int N, int K,
    std::vector<std::vector<int>>& lists
) {
    lists.assign(K, {});
    for (int i = 0; i < N; ++i) {
        int c = assign[i];
        if (c >= 0 && c < K) {
            lists[c].push_back(i);
        }
    }
}

// ---------------- Main ----------------
int main() {
    cudaDeviceProp prop;
    cudaGetDeviceProperties(&prop, 0);

    printf("Params: N=%d  K=%d  DIM=%d  BLOCK=%d\n", N, K, DIM, BLOCK);
    printf("Mode: GPU Training V3.0 (No Unroll Pragma)\n");

    initBases();
    std::mt19937 rng(SEED + 7);

    // 1) Data Generation
    std::vector<int>  h_cards((size_t)N * 25);
    std::vector<float> h_data((size_t)N * DIM);
    printf("[INIT] Generating %d vectors...\n", N);
    
    for (int i = 0; i < N; i++) {
        int c[25]; genCard(rng, c);
        for (int t = 0; t < 25; ++t) h_cards[(size_t)i * 25 + t] = c[t];
        Vec v = cardToVec(c);
        for (int d = 0; d < DIM; ++d) h_data[(size_t)i * DIM + d] = v[d];
    }

    // 2) Build Query
    int qc[25];
    for (int t = 0; t < 25; ++t) qc[t] = h_cards[t]; 
    qc[3] = 75; qc[17] = 1; std::swap(qc[5], qc[19]); 
    Vec qvec = cardToVec(qc);
    std::vector<float> q_host(qvec.begin(), qvec.end());

    {
        double dot0 = dot_host(q_host.data(), &h_data[0]);
        printf("[DEBUG] Query vs Data[0]: dot=%.9f dist=%.9f\n", dot0, 1.0 - dot0);
    }

    // 3) GPU Allocations
    float *d_data = 0, *d_centroids = 0;
    int   *d_assign = 0;

    // Final Buffers
    double *d_sums_final = 0; 
    int    *d_counts_final = 0;
    
    // Temporary Block Buffers
    double *d_block_sums = 0;
    int    *d_block_counts = 0;

    // Grid Calculation
    dim3 block(BLOCK);
    dim3 gridN((N + BLOCK - 1) / BLOCK);
    int GridSize = gridN.x;
    
    // Determine kernel launch dim for reduction
    dim3 gridReduce((K * DIM + BLOCK - 1) / BLOCK);
    
    // Determine kernel launch dim for update
    dim3 gridUpdate((K + BLOCK - 1) / BLOCK);

    size_t szBlockSums   = (size_t)GridSize * K * DIM * sizeof(double);
    size_t szBlockCounts = (size_t)GridSize * K * sizeof(int);

    cudaMalloc(&d_data,      (size_t)N * DIM * sizeof(float));
    cudaMalloc(&d_centroids, (size_t)K * DIM * sizeof(float));
    cudaMalloc(&d_assign,    N * sizeof(int));
    
    cudaMalloc(&d_sums_final,   (size_t)K * DIM * sizeof(double));
    cudaMalloc(&d_counts_final, K * sizeof(int));
    
    cudaMalloc(&d_block_sums,   szBlockSums);
    cudaMalloc(&d_block_counts, szBlockCounts);

    // Copy Data
    cudaMemcpy(d_data, h_data.data(), (size_t)N * DIM * sizeof(float), cudaMemcpyHostToDevice);

    // 4) Initialize Centroids
    {
        std::vector<int> idx(N);
        std::iota(idx.begin(), idx.end(), 0);
        std::shuffle(idx.begin(), idx.end(), rng);
        std::vector<float> h_initC((size_t)K * DIM);
        for (int c = 0; c < K; ++c) {
            int i = idx[c];
            std::copy_n(&h_data[(size_t)i * DIM], DIM, &h_initC[(size_t)c * DIM]);
        }
        cudaMemcpy(d_centroids, h_initC.data(), (size_t)K * DIM * sizeof(float), cudaMemcpyHostToDevice);
    }

    // 5) GPU K-Means Training Loop
    printf("[TRAIN] Starting V3.0 K-Means (%d iters)...\n", KMEANS_ITERS);
    
    for (int it = 0; it < KMEANS_ITERS; ++it) {
        // A. Clear Block Accumulators
        cudaMemset(d_block_sums,   0, szBlockSums);
        cudaMemset(d_block_counts, 0, szBlockCounts);
        
        // B. E-Step
        assignAndAccumulatePerBlockKernel_V3<<<gridN, block>>>(
            d_data, N, d_centroids, K, d_assign, d_block_sums, d_block_counts
        );
        
        // C. Reduction
        reduceSumsKernel<<<gridReduce, block>>>(
            d_sums_final, d_counts_final, d_block_sums, d_block_counts, K, GridSize
        );
        
        // D. M-Step
        updateCentroidsKernel<<<gridUpdate, block>>>(
            d_centroids, d_sums_final, d_counts_final, K
        );
        cudaDeviceSynchronize();
    }
    printf("[TRAIN] Done.\n");

    // 6) Retrieve Results
    std::vector<int> h_assign(N);
    cudaMemcpy(h_assign.data(), d_assign, N * sizeof(int), cudaMemcpyDeviceToHost);

    std::vector<float> h_finalCentroids(K * DIM);
    cudaMemcpy(h_finalCentroids.data(), d_centroids, (size_t)K * DIM * sizeof(float), cudaMemcpyDeviceToHost);

    // Cleanup GPU
    cudaFree(d_data); cudaFree(d_centroids); cudaFree(d_assign);
    cudaFree(d_sums_final); cudaFree(d_counts_final);
    cudaFree(d_block_sums); cudaFree(d_block_counts);

    // 7) Build Host Index
    printf("[INDEX] Building Inverted Lists...\n");
    std::vector<std::vector<int>> lists;
    buildInvertedLists(h_assign, N, K, lists);

    int nonEmpty = 0;
    for(const auto& list : lists) if(!list.empty()) nonEmpty++;
    printf("[INDEX] Non-empty clusters: %d / %d\n", nonEmpty, K);
    
    if (nonEmpty < K * 0.1) {
        printf("WARNING: Too many empty clusters! Check initialization or data distribution.\n");
    }

    // 8) CPU Search
    printf("\n[SEARCH] CPU Search (nprobe=%d)...\n", NPROBE);

    // Coarse Search
    std::vector<std::pair<float, int>> centerDists;
    centerDists.reserve(K);
    for (int c = 0; c < K; ++c) {
        const float* ctr = &h_finalCentroids[(size_t)c * DIM];
        double dotVal = dot_host(q_host.data(), ctr);
        float dist = 1.0f - (float)dotVal;
        centerDists.push_back({dist, c});
    }
    std::sort(centerDists.begin(), centerDists.end(), 
              [](const std::pair<float, int>& a, const std::pair<float, int>& b){
                  return a.first < b.first;
              });

    // Fine Search
    std::vector<std::pair<float, int>> candidates;
    candidates.reserve((N / K) * NPROBE * 2);

    int visitedVecs = 0;
    for (int i = 0; i < NPROBE && i < K; ++i) {
        int c_id = centerDists[i].second;
        const auto& bucket = lists[c_id];
        visitedVecs += bucket.size();

        for (int vecIdx : bucket) {
            const float* vec = &h_data[(size_t)vecIdx * DIM];
            double dotVal = dot_host(q_host.data(), vec);
            float dist = 1.0f - (float)dotVal;
            candidates.push_back({dist, vecIdx});
        }
    }
    printf("[SEARCH] Scanned %d vectors.\n", visitedVecs);

    // Top-K
    if (candidates.empty()) {
        printf("No candidates found.\n");
    } else {
        int finalK = std::min((int)candidates.size(), TOPK);
        std::partial_sort(candidates.begin(), 
                          candidates.begin() + finalK, 
                          candidates.end(),
                          [](const std::pair<float, int>& a, const std::pair<float, int>& b){
                              return a.first < b.first;
                          });

        printf("\nTop-%d Results:\n", finalK);
        for (int i = 0; i < finalK; ++i) {
            printf("%2d) id=%d  dist=%.6f  sim=%.6f\n",
                   i+1, candidates[i].second, candidates[i].first, 1.f - candidates[i].first);
        }
    }

    return 0;
}

Overwriting kmean_gpu_share_mem.cu


In [21]:
%%bash
nvcc -O3 -std=c++17 -arch=sm_70 kmean_gpu_share_mem.cu -o kmean_gpu_share_mem -Wno-deprecated-gpu-targets

In [22]:
%%bash
nvprof ./kmean_gpu_share_mem

==386315== NVPROF is profiling process 386315, command: ./kmean_gpu_share_mem


Params: N=1048576  K=2048  DIM=64  BLOCK=512
Mode: GPU Training V3.0 (No Unroll Pragma)
[INIT] Generating 1048576 vectors...
[DEBUG] Query vs Data[0]: dot=0.927471359 dist=0.072528641
[TRAIN] Starting V3.0 K-Means (15 iters)...
[TRAIN] Done.
[INDEX] Building Inverted Lists...
[INDEX] Non-empty clusters: 2048 / 2048

[SEARCH] CPU Search (nprobe=32)...
[SEARCH] Scanned 18350 vectors.

Top-5 Results:
 1) id=0  dist=0.072529  sim=0.927471
 2) id=89698  dist=0.157913  sim=0.842087
 3) id=913782  dist=0.158968  sim=0.841032
 4) id=350035  dist=0.164414  sim=0.835586
 5) id=56801  dist=0.177664  sim=0.822336


==386315== Profiling application: ./kmean_gpu_share_mem
==386315== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   68.68%  703.20ms        15  46.880ms  42.476ms  50.344ms  assignAndAccumulatePerBlockKernel_V3(float const *, int, float const *, int, int*, double*, int*)
                   23.27%  238.22ms         2  119.11ms  93.184us  238.12ms  [CUDA memcpy HtoD]
                    4.02%  41.109ms        15  2.7406ms  2.7238ms  2.7783ms  reduceSumsKernel(double*, int*, double const *, int const *, int, int)
                    3.55%  36.336ms        30  1.2112ms  20.448us  2.4143ms  [CUDA memset]
                    0.28%  2.8982ms         2  1.4491ms  125.47us  2.7727ms  [CUDA memcpy DtoH]
                    0.20%  2.0625ms        15  137.50us  133.12us  150.50us  updateCentroidsKernel(float*, double const *, int const *, int)
      API calls:   49.49%  1.01629s         7  145.18ms  5.6800us  1.01428s  cudaMallo